In [2]:
import pymongo
import operator
import numpy as np
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
db = client.nesoi
terrain = db.terrain

In [4]:
def createCType():
    """Create a sorted list of unique Terrain."""
    
    c = set()
    for cell in terrain.find():
        c.add(cell["terrain"])

    tmp = list(c)
    
    return sorted(tmp)

In [5]:
CType = createCType()
print CType

[u'bridge', u'bridgeGreen', u'bushGreen', u'bushRed', u'bushWhite', u'cave', u'dirt', u'dirtGrey', u'gateGreenCyclopsTop', u'gateGreenLeft', u'gateGreenRight', u'gateGreenTop', u'gateGreenTopLeft', u'gateGreenTopRight', u'gateRedLeft', u'gateRedRight', u'gateRedTop', u'gateRedTopLeft', u'gateRedTopRight', u'gateWhiteCyclopsTop', u'gateWhiteLeft', u'gateWhiteRight', u'gateWhiteTop', u'gateWhiteTopLeft', u'gateWhiteTopRight', u'guardGreen', u'guardRed', u'guardWhite', u'headstone', u'ladderGreen', u'ladderRed', u'ladderWhite', u'mountainGreen', u'mountainGreenLowerLeft', u'mountainGreenLowerRight', u'mountainGreenTop', u'mountainGreenTopLeft', u'mountainGreenTopRight', u'mountainRed', u'mountainRedLowerLeft', u'mountainRedLowerRight', u'mountainRedTop', u'mountainRedTopLeft', u'mountainRedTopRight', u'mountainWhite', u'mountainWhiteLowerLeft', u'mountainWhiteLowerRight', u'mountainWhiteTop', u'mountainWhiteTopLeft', u'mountainWhiteTopRight', u'rockGreen', u'rockRed', u'sand', u'sandRedLe

In [6]:
def cTypeToIdx(val):
    """Return the index of a given terrain value. Throws a ValueError if the value is not found in the list."""
    
    return CType.index(val)

In [7]:
def getCellIdxTerrainDistribution(idx, threshold, show):
    """
    Return the distribution of a given index and threshold and normalized distribution over the new subset.
    
    Args:
        idx (int): The cell index of a page. The range is 0-175. 0 is the upper left corner.
        threshold (int): Represents the total of the most common values in the distribution.
        show (bool): If true, returns a subset of the distribution for the given threshold
    
    Example: 
        For the distribution [a:50, b:20, c:10, d:8, e:7, f:3, g:2] and show is true
        Threshold 50 would return [a:100]
        Threshold 70 would return [a:71, b:29]
        Threshold 80 would return [a:63, b:26, c:13]
        Threshold 90 would return [a:56, b:22, c:11, d:9, e:8]
        Threshold 100 would return all items in the distributions
    
    """

    d = getDistribution(idx)
    
    if show:
        sorted_x = getThreshold(sorted_x, threshold)
    
    return sorted_x

In [8]:
def getThreshold(d, threshold):
    """
    Returns the top most common items in a distribution and normalized distribution over the new subset.
    
    Args:
        d (array): The array of distribution.
        threshold (int): Represents the total of the most common values in the distribution.
       
    Example: 
        For the distribution [a:50, b:20, c:10, d:8, e:7, f:3, g:2] 
        Threshold 50 would return [a:100]
        Threshold 70 would return [a:71, b:29]
        Threshold 80 would return [a:63, b:26, c:13]
        Threshold 90 would return [a:56, b:22, c:11, d:9, e:8]
        Threshold 100 would return all items in the distributions
    """
    
    accum = 0
    terrains = ()
    cells = {}
    for ter, per in d:
        cells[ter] = per

        accum = accum + per  

        if accum > threshold:
            break

    for key in cells:
        cells[key] = cells[key] / accum
    
    sorted_x = sorted(cells.items(), key=operator.itemgetter(1), reverse=True) 
    return sorted_x

In [9]:
def getDistribution(idx):
    """
    Return the distribution of a given index. 
    
    Args:
        idx (int): The cell index of a page. The range is 0-175. 0 is the upper left corner.
    """
    
    # Get all of the cells for this index
    results = list()
    for cell in terrain.find({"idx":idx}):
        results.append(cell["terrain"])

    # Calculate the Distribution of values
    uniqueItems = set(results)
    totalLen = float(len(results)) 
    d = {}
    for item in uniqueItems:
        percent = 100*results.count(item) / totalLen
        d[item] = percent
        
    # Sort by Terrain type
    return sorted(d.items(), key=operator.itemgetter(1), reverse=True)


In [10]:
distZero = getDistribution(0)
print distZero
total = 0
for t,p in distZero:
    total = total + p
    print cTypeToIdx(t), p
    print total

[(u'mountainRed', 50.78125), (u'bushGreen', 14.0625), (u'mountainGreen', 10.9375), (u'bushRed', 7.8125), (u'mountainWhite', 6.25), (u'waterRed', 5.46875), (u'waterGreen', 4.6875)]
38 50.78125
50.78125
2 14.0625
64.84375
32 10.9375
75.78125
3 7.8125
83.59375
44 6.25
89.84375
83 5.46875
95.3125
72 4.6875
100.0


In [11]:
distOne = getDistribution(1)
print distOne

[(u'mountainRed', 51.93798449612403), (u'bushGreen', 12.4031007751938), (u'mountainGreen', 10.852713178294573), (u'bushRed', 7.751937984496124), (u'mountainWhite', 6.2015503875969), (u'waterGreen', 4.651162790697675), (u'waterRed', 3.10077519379845), (u'dirt', 2.3255813953488373), (u'sand', 0.7751937984496124)]


In [9]:
distTop70ofZero = getThreshold(distZero, 60)
print distTop70ofZero
total = 0
for t,p in distTop70ofZero:
    total = total + p
    print cTypeToIdx(t), p
    print total

NameError: name 'distZero' is not defined

In [22]:
def generateCss(idx, threshold):
    
    print ".content div:nth-child(" + str(idx + 1) + ")" + ".t" + str(threshold) + " {animation-name: bg" + str(idx) + "-t" + str(threshold) + ";}"

    curPercent = 0
    firstTerrain = ""

    css = "@keyframes bg" + str(idx) + "-t" + str(threshold) + " {"
    
    d = getCellIdxTerrainDistribution(idx, threshold, True)

    for ter, per in d:
        
        per = per * 100

        if not firstTerrain:
            firstTerrain = ter
            css = css + "0% {background-image: url('img/" + ter + ".png');}"
            css = css + "{0:.0f}".format(per/ 2) + "% {background-image: url('img/" + ter + ".png');}"
        else:
            css = css + "{0:.0f}".format(curPercent) + "% {background-image: url('img/" + ter + ".png');}"
            
        curPercent = curPercent + per

    css = css + "100% {background-image: url('img/" + firstTerrain + ".png');}}"

    print css

In [94]:
for i in range(0,1):
    generateCss(i, 100)
    generateCss(i, 90)
    generateCss(i, 80)
    generateCss(i, 70)
    generateCss(i, 60)
    generateCss(i, 50)
    generateCss(i, 50)
    generateCss(i, 40)
    generateCss(i, 30)
    generateCss(i, 20)
    generateCss(i, 10)
    

.content div:nth-child(1).t100 {animation-name: bg0-t100;}
@keyframes bg0-t100 {0% {background-image: url('img/mountainRed.png');}25% {background-image: url('img/mountainRed.png');}51% {background-image: url('img/bushGreen.png');}65% {background-image: url('img/mountainGreen.png');}76% {background-image: url('img/bushRed.png');}84% {background-image: url('img/mountainWhite.png');}90% {background-image: url('img/waterRed.png');}95% {background-image: url('img/waterGreen.png');}100% {background-image: url('img/mountainRed.png');}}
.content div:nth-child(1).t90 {animation-name: bg0-t90;}
@keyframes bg0-t90 {0% {background-image: url('img/mountainRed.png');}27% {background-image: url('img/mountainRed.png');}53% {background-image: url('img/bushGreen.png');}68% {background-image: url('img/mountainGreen.png');}80% {background-image: url('img/bushRed.png');}88% {background-image: url('img/mountainWhite.png');}94% {background-image: url('img/waterRed.png');}100% {background-image: url('img/moun

In [348]:
# # Look at the pair combinatons 
# def getPairs():
#     pairs = list()
#     for cell in terrain.find({"idx":0}):

#         x = cell["x"]
#         y = cell["y"] 

#         pair = [(x, y, cell["terrain"])]

#         for neighbor in terrain.find({"x":x + 1, "y": y}):
#             pair.append((neighbor["x"], neighbor["y"], neighbor["terrain"]))

#         pairs.append(pair)
        
#     return pairs

In [349]:
# pairs = getPairs();
# pairs

In [350]:
# # d={'a':[],'b':[]}
# d={}
# for p in pairs:
 
#     if(not p[0][2] in d.keys()):
#         d[p[0][2]] = list()

#     d[p[0][2]].append(p[1][2])

In [351]:
# d

In [39]:
# terra = list()
# total = 0
# for i in d:
#     cellTerrainItemCount = {}
   
#     for j in d[i]:
       
#         if(not j in cellTerrainItemCount.keys()):
#             cellTerrainItemCount[j] = 0
        
#         cellTerrainItemCount[j] += 1
       
     
#     terra.append((i, cellTerrainItemCount))
    

In [352]:
# terra


# Helper Functions

In [12]:
def idxToCoord(idx): 
    x = idx % 16
    y = idx / 16
    return x, y  

In [13]:
def removeZeroColumnsRows(matrix):
    
    if isinstance(matrix, np.matrix) :
        rows = (matrix).any(axis=1)
        nx = matrix[rows.A1]
        cols = (matrix).any(axis=0)
        ny = nx[:,cols.A1]
    elif isinstance(matrix, np.ndarray) :
        # Remove any zero rows
        nx = matrix[(matrix).any(axis=1)]
        ny = nx[:,(nx).any(axis=0)]
    else:
        ny = np.zeros(len(matrix))
        
    return ny

# Conditional Probability Distribution 

The goal is to predict what the Cell terrain will be in index 1 given the value in index 0. 

`P(Y | X)`. Where `X` = Cell index 0 and `Y` = Cell index 1

\begin{equation*}
P(Y|X) = P(X) * P(Y | X)
\end{equation*}

In [14]:
def createObservationDisribution(idx1, idx2):
    
    # Create an empty matrix of zeros 
    od = np.zeros([len(CType),len(CType)])
    
    for cell in terrain.find({"idx":idx1}):

        # Find the second index for this cell on this page 
        x = cell["x"]
        y = cell["y"] 
        
        delX, delY = idxToCoord(idx2)
        
        cell2 = terrain.find_one({"x":x + delX, "y":y + delY})
        
        # Fetch the terrain
        Xi = cTypeToIdx(cell["terrain"])            
        Yi = cTypeToIdx(cell2["terrain"])
            
        od[Xi, Yi] += 1
    
    return od

In [15]:
def createJointDisribution(od):
    
    # Find the total size
    size = od.size
    
    # Calculate the Distribution of values
    uniqueItems = set(results)
    totalLen = float(len(results)) 
    d = {}
    for item in uniqueItems:
        percent = 100*results.count(item) / totalLen
        d[item] = percent

In [16]:
od = createObservationDisribution(0,1)

In [17]:
od / od.size

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.00159422, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
size = 9409
for row in od:
    for c in row:
        
        print c, 
    print ""
    
print size

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 15.0 0.0 0.0 0.0 3.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0

In [18]:
def getMarginalDistribution(idx):
    od = np.zeros([len(CType)])
    
    for cell in terrain.find({"idx":idx}):
            
        Xi = cTypeToIdx(cell["terrain"])
        od[Xi] += 1
    
    return od

In [19]:
print removeZeroColumnsRows(od)

[[ 15.   0.   3.   0.   0.   0.   0.   0.   0.]
 [  0.  10.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.  14.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  64.   0.   1.   0.   0.]
 [  0.   0.   0.   0.   0.   8.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   6.   0.]
 [  0.   0.   0.   0.   3.   0.   0.   0.   4.]]


In [22]:
marginal0 = getMarginalDistribution(0)
print marginal0
marginal0.sum()

[  0.   0.  18.  10.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.  14.   0.   0.   0.   0.   0.  65.   0.   0.   0.   0.   0.   8.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   6.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   7.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.]


128.0

In [23]:
marginal1 = getMarginalDistribution(1)
print marginal1
marginal1.sum()

# Error: There should only be 15 bushGreen for Cell1
# And only 128 count

[  0.   0.  16.  10.   0.   0.   3.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.  14.   0.   0.   0.   0.   0.  67.   0.   0.   0.   0.   0.   8.
   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   6.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   4.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.]


129.0

In [55]:
print (marginal0)
marginal = marginal0 / np.sum(marginal0)

[  0.   0.  18.  10.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.  14.   0.   0.   0.   0.   0.  65.   0.   0.   0.   0.   0.   8.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   6.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   7.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.]


In [63]:
printJsArray(marginal)

'[0.0, 0.0, 0.140625, 0.078125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.109375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5078125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.046875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0546875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.]'

In [153]:
def getCellMarginals():
    marginals = []

    for i in range(0, 176):
        marginal = getMarginalDistribution(i)
        marginal = marginal / np.sum(marginal)

        marginals.append(marginal)
    
    return np.array(marginals)

In [154]:

marginals = getCellMarginals()


In [155]:
printJsArray(marginals)


'[[0.0, 0.0, 0.140625, 0.078125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.109375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5078125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.046875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0546875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.124031007752, 0.077519379845, 0.0, 0.0, 0.0232558139535, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.108527131783, 0.0, 0.0, 0.0, 0.0, 0.0, 0.519379844961, 0.0, 0.0, 0.0, 0.0, 0.0, 0.062015503876, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0077519379845, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.046511627907, 0.0, 0.0, 0.0, 0.0, 0

In [37]:
def validateObservedResults(od):
    # Validate expected counts

    print od[cTypeToIdx('mountainGreen'),cTypeToIdx('mountainGreen')] == 14
    print od[cTypeToIdx('waterGreen'),cTypeToIdx('waterGreen')] == 6
    print od[cTypeToIdx('waterRed'),cTypeToIdx('waterRed')] == 4
    print od[cTypeToIdx('waterRed'), cTypeToIdx('mountainRed')] == 3
    print od[cTypeToIdx('mountainWhite'),cTypeToIdx('mountainWhite')] == 8
    print od[cTypeToIdx('mountainRed'),cTypeToIdx('mountainRed')] == 64
    print od[cTypeToIdx('mountainRed'), cTypeToIdx('sand')] == 1
    print od[cTypeToIdx('bushRed'),cTypeToIdx('bushRed')] == 10
    print od[cTypeToIdx('bushGreen'), cTypeToIdx('bushGreen')] == 15
    print od[cTypeToIdx('bushGreen'),cTypeToIdx('dirt')] == 3
    print od.sum() == 128

validateObservedResults(od)

True
True
True
True
True
True
True
True
True
True
True


In [38]:
def createCpd(observedDistribution):
    rowSum = np.sum(observedDistribution,1) + 0.0000000001
    cpd = np.divide(observedDistribution, np.matrix(rowSum).T)[:None]
    return cpd

In [39]:
cpd = createCpd(od)

In [41]:
smallCpd = removeZeroColumnsRows(np.around(cpd, decimals=3))
print smallCpd

[[ 0.833  0.     0.167  0.     0.     0.     0.     0.     0.   ]
 [ 0.     1.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     1.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.985  0.     0.015  0.     0.   ]
 [ 0.     0.     0.     0.     0.     1.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     1.     0.   ]
 [ 0.     0.     0.     0.     0.429  0.     0.     0.     0.571]]


In [42]:
getCellIdxTerrainDistribution(0,100,False)

[(u'mountainRed', 50.78125),
 (u'bushGreen', 14.0625),
 (u'mountainGreen', 10.9375),
 (u'bushRed', 7.8125),
 (u'mountainWhite', 6.25),
 (u'waterRed', 5.46875),
 (u'waterGreen', 4.6875)]

In [45]:
print getDistribution(0)
marginal = np.zeros(len(CType))

for t,p in getDistribution(0):
    marginal[cTypeToIdx(t)] = p

print marginal

[(u'mountainRed', 50.78125), (u'bushGreen', 14.0625), (u'mountainGreen', 10.9375), (u'bushRed', 7.8125), (u'mountainWhite', 6.25), (u'waterRed', 5.46875), (u'waterGreen', 4.6875)]
[  0.        0.       14.0625    7.8125    0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.        0.
  10.9375    0.        0.        0.        0.        0.       50.78125   0.
   0.        0.        0.        0.        6.25      0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.        0.
   4.6875    0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        5.46875   0.        0.     

In [120]:
def printJsArray(matrix):
    
    
    jsStr = "["
    
    if isinstance(matrix, np.matrix) :
        matrix = np.asarray(matrix)
        for row in matrix:

            jsStr = jsStr + "["
            for col in row:
                
                jsStr = jsStr + str(col).rstrip() + ", "
            
            jsStr = jsStr[:-2]
            jsStr = jsStr + "],"
            
       
        
        
    elif isinstance(matrix, np.ndarray) :
        

        
        if(len(matrix.shape) == 1):
            
         
            
            for item in matrix:
                 jsStr = jsStr + str(item).strip() + ", "
            jsStr = jsStr[:-2]
            
        else:
            
         
            
            for row in matrix:
                jsStr = jsStr + "["
                for col in row:
                    jsStr = jsStr + str(col).strip() + ", "

                jsStr = jsStr[:-2]
                jsStr = jsStr + "],"
            
       
    else:
       
        for i in matrix:
            jsStr = jsStr + str(i) + ","
            
    jsStr = jsStr[:-1]
    jsStr = jsStr + "]"
        
    return jsStr.replace('\n','')

In [47]:
printJsArray(cpd)

'[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.833333333329,

In [19]:
cTypeToIdx("waterGreenTopRight")

ValueError: 'waterGreenTopRight' is not in list

In [36]:
printJsCType()

[ 'bridge', 'bridgeGreen', 'bushGreen', 'bushRed', 'bushWhite', 'cave', 'dirt', 'dirtGrey', 'gateGreenCyclopsTop', 'gateGreenLeft', 'gateGreenRight', 'gateGreenTop', 'gateGreenTopLeft', 'gateGreenTopRight', 'gateRedLeft', 'gateRedRight', 'gateRedTop', 'gateRedTopLeft', 'gateRedTopRight', 'gateWhiteCyclopsTop', 'gateWhiteLeft', 'gateWhiteRight', 'gateWhiteTop', 'gateWhiteTopLeft', 'gateWhiteTopRight', 'guardGreen', 'guardRed', 'guardWhite', 'headstone', 'ladderGreen', 'ladderRed', 'ladderWhite', 'mountainGreen', 'mountainGreenLowerLeft', 'mountainGreenLowerRight', 'mountainGreenTop', 'mountainGreenTopLeft', 'mountainGreenTopRight', 'mountainRed', 'mountainRedLowerLeft', 'mountainRedLowerRight', 'mountainRedTop', 'mountainRedTopLeft', 'mountainRedTopRight', 'mountainWhite', 'mountainWhiteLowerLeft', 'mountainWhiteLowerRight', 'mountainWhiteTop', 'mountainWhiteTopLeft', 'mountainWhiteTopRight', 'rockGreen', 'rockRed', 'sand', 'sandRedLeft', 'sandRedLower', 'sandRedLowerLeft', 'sandRedLowe

In [35]:
def printJsCType():
    print "[",
    for i in CType:
        print  "'" + i + "',",
    print "]"

In [32]:
CType

[u'bridge',
 u'bridgeGreen',
 u'bushGreen',
 u'bushRed',
 u'bushWhite',
 u'cave',
 u'dirt',
 u'dirtGrey',
 u'gateGreenCyclopsTop',
 u'gateGreenLeft',
 u'gateGreenRight',
 u'gateGreenTop',
 u'gateGreenTopLeft',
 u'gateGreenTopRight',
 u'gateRedLeft',
 u'gateRedRight',
 u'gateRedTop',
 u'gateRedTopLeft',
 u'gateRedTopRight',
 u'gateWhiteCyclopsTop',
 u'gateWhiteLeft',
 u'gateWhiteRight',
 u'gateWhiteTop',
 u'gateWhiteTopLeft',
 u'gateWhiteTopRight',
 u'guardGreen',
 u'guardRed',
 u'guardWhite',
 u'headstone',
 u'ladderGreen',
 u'ladderRed',
 u'ladderWhite',
 u'mountainGreen',
 u'mountainGreenLowerLeft',
 u'mountainGreenLowerRight',
 u'mountainGreenTop',
 u'mountainGreenTopLeft',
 u'mountainGreenTopRight',
 u'mountainRed',
 u'mountainRedLowerLeft',
 u'mountainRedLowerRight',
 u'mountainRedTop',
 u'mountainRedTopLeft',
 u'mountainRedTopRight',
 u'mountainWhite',
 u'mountainWhiteLowerLeft',
 u'mountainWhiteLowerRight',
 u'mountainWhiteTop',
 u'mountainWhiteTopLeft',
 u'mountainWhiteTopRight